## Electre Iv / Is

```python
from EasyMCDM.models.Electre import Electre

data = {
    "A1" : [80, 90,  600, 5.4,  8,  5],
    "A2" : [65, 58,  200, 9.7,  1,  1],
    "A3" : [83, 60,  400, 7.2,  4,  7],
    "A4" : [40, 80, 1000, 7.5,  7, 10],
    "A5" : [52, 72,  600, 2.0,  3,  8],
    "A6" : [94, 96,  700, 3.6,  5,  6],
}
weights = [0.1, 0.2, 0.2, 0.1, 0.2, 0.2]
prefs = ["min", "max", "min", "min", "min", "max"]
vetoes = [45, 29, 550, 6, 4.5, 4.5]
indifference_threshold = 0.6
preference_thresholds = [20, 10, 200, 4, 2, 2] # or None for Electre Iv

e = Electre(data=data, verbose=False)

results = e.solve(weights, prefs, vetoes, indifference_threshold, preference_thresholds)
```

**Output :**

```python
{'kernels': ['A4', 'A5']}
```

In [98]:
import kotlin.math.abs


class Electre(
    private val criteria: List<List<Double>>,
    private val verbosePrintOutput: Boolean = false
) {

    private fun printElectre(
        concordanceMatrix: List<DoubleArray>,
        nonDiscordanceMatrix: List<DoubleArray>,
        outrankingMatrix: List<Array<Boolean?>>,
        kernels: List<String>,
        robustnessAnalysisResults: List<String>,
        frequentKernels: List<String>
    ) {

        println("concordanceMatrix: ")
        concordanceMatrix.forEach { row ->
            println(row.joinToString("\t\t") { "%.3f".format(it) })
        }
        println()

        println("nonDiscordanceMatrix: ")
        nonDiscordanceMatrix.forEach { row ->
            println(row.joinToString("\t\t") { "%.3f".format(it) })
        }
        println()

        println("outrankingMatrix: ")
        outrankingMatrix.forEach { row ->
            println(row.joinToString("\t\t"))
        }
        println()


        println(kernels)

        println("Robustness Analysis:")
        robustnessAnalysisResults.forEach { println(it) }

        println()

        println("Most Frequent Kernels:")
        println(frequentKernels.joinToString(", "))
    }


    /**
     * Perform robustness analysis on the concordance and non-discordance matrices
     * @param concordanceMatrix the concordance matrix
     * @param nonDiscordanceMatrix the non-discordance matrix
     * @return the robustness analysis results
     */
    private fun robustnessAnalysis(
        concordanceMatrix: List<DoubleArray>,
        nonDiscordanceMatrix: List<DoubleArray>
    ): Pair<List<String>, List<String>> {

        val robustnessAnalysisKernels = mutableMapOf<Double, List<String>>()

        generateSequence(0.5) { it + 0.025 }
            .takeWhile { it <= 1.0 }
            .forEach { threshold ->
                val outrankingMatrix = getOutrankingMatrix(
                    concordanceMatrix, nonDiscordanceMatrix, threshold
                )

                robustnessAnalysisKernels[threshold] = getKernels(outrankingMatrix)
            }
        
        val analysisResults = mutableListOf<String>()
        val occurrences = mutableMapOf<String, Int>()

        for ((key, value) in robustnessAnalysisKernels) {
            value.forEach {
                occurrences[it] = occurrences.getOrDefault(it, 1) + 1
            }
            analysisResults.add("[%.3f] = %s".format(key, value.joinToString(", ")))
        }

        val orderedCandidates = occurrences.entries
            .sortedByDescending { it.value }
            .map { "${it.key} (${it.value})" }

        return analysisResults to orderedCandidates
    }


    /**
     * Get the concordance and non-discordance matrices
     * @param weights the weights for each criterion
     * @param prefs the preferences for each criterion
     * @param vetoes the vetoes for each criterion
     * @param preferenceThresholds the preference thresholds for each criterion (optional for ELECTRE I-s)
     * @return the concordance and non-discordance matrices
     */
    private fun getElectre1Matrices(
        weights: List<Double>,
        prefs: List<String>,
        vetoes: List<Double>,
        preferenceThresholds: List<Double>? = null
    ): Pair<List<DoubleArray>, List<DoubleArray>> {

        val concordanceMatrix = List(criteria.size) { DoubleArray(criteria.size) { 0.0 } }
        val nonDiscordanceMatrix = List(criteria.size) { DoubleArray(criteria.size) { 0.0 } }

        for (x in criteria.indices) {
            for (y in criteria.indices) {
                if (x == y) {
                    concordanceMatrix[x][y] = Double.NaN
                    nonDiscordanceMatrix[x][y] = Double.NaN
                    continue
                }

                val (a, b) = criteria[x] to criteria[y]
                var (av, bv) = 0.0 to 0.0

                var (aRespectsVetoes, bRespectsVetoes) = true to true


                for (idx in weights.indices) {
                    val (w, p, v) = Triple(weights[idx], prefs[idx], vetoes[idx])

                    val bestVal = if (p == "max") {
//                        max(a[idx], b[idx])
                        a[idx].coerceAtLeast(b[idx])
                    } else {
//                        min(a[idx], b[idx])
                        a[idx].coerceAtMost(b[idx])
                    }

                    val diff = abs(b[idx] - a[idx])

                    // NOTE: ELECTRE I-v
                    var points = if (diff != 0.0) 0.0 else w

                    // NOTE: ELECTRE I-s
                    if (preferenceThresholds != null) {
                        val prefThreshold = preferenceThresholds[idx]
                        if (diff < prefThreshold) {
                            points = (1 - (diff / prefThreshold)) * w
                        }
                    }

                    if (bestVal == a[idx]) {
                        av += w
                        bv += points
                        bRespectsVetoes = bRespectsVetoes && diff < v
                    } else {
                        av += points
                        bv += w
                        aRespectsVetoes = aRespectsVetoes && diff < v
                    }
                }

                concordanceMatrix[x][y] = av
                nonDiscordanceMatrix[x][y] = if (aRespectsVetoes) 1.0 else 0.0

                concordanceMatrix[y][x] = bv
                nonDiscordanceMatrix[y][x] = if (bRespectsVetoes) 1.0 else 0.0
            }
        }

        return concordanceMatrix to nonDiscordanceMatrix
    }

    /**
     * Get the outranking matrix
     * @param concordanceMatrix the concordance matrix
     * @param nonDiscordanceMatrix the non-discordance matrix
     * @param concordanceThreshold the concordance threshold
     * @return the outranking matrix
     */
    private fun getOutrankingMatrix(
        concordanceMatrix: List<DoubleArray>,
        nonDiscordanceMatrix: List<DoubleArray>,
        concordanceThreshold: Double
    ): List<Array<Boolean?>> {

        val size = criteria.size
        val outrankingMatrix = List(size) { Array<Boolean?>(size) { false } }

        for (x in criteria.indices) {
            for (y in criteria.indices) {
                if (x == y) {
                    outrankingMatrix[x][y] = null
                    continue
                }

                val ac = concordanceMatrix[x][y]
                val av = nonDiscordanceMatrix[x][y]
                val bc = concordanceMatrix[y][x]
                val bv = nonDiscordanceMatrix[y][x]

                outrankingMatrix[x][y] = ac > concordanceThreshold && av != 0.0 && !av.isNaN()
                outrankingMatrix[y][x] = bc > concordanceThreshold && bv != 0.0 && !bv.isNaN()
            }
        }

        return outrankingMatrix
    }

    /**
     * Get the kernels from the outranking matrix
     * @param resultMatrix the outranking matrix
     * @return the kernels
     * @see getOutrankingMatrix
     */
    private fun getKernels(resultMatrix: List<Array<Boolean?>>) =
        criteria.mapIndexedNotNull { col, _ ->
            var isKernel = true
            for (row in criteria.indices) {
                if (resultMatrix[row][col] != null && resultMatrix[row][col] == true) {
                    isKernel = false
                    break
                }
            }

            if (isKernel) (col + 1).toString()
            else null
        }

    /**
     * Solve the ELECTRE problem
     * @param weights the weights for each criterion
     * @param prefs the preferences for each criterion
     * @param vetoes the vetoes for each criterion
     * @param concordanceThreshold the concordance threshold
     * @param preferenceThresholds the preference thresholds for each criterion (optional for ELECTRE I-s)
     * @return the kernels and frequent kernels
     */
    fun solve(
        weights: List<Double>,
        prefs: List<String>,
        vetoes: List<Double>,
        concordanceThreshold: Double,
        preferenceThresholds: List<Double>? = null
    ): Map<String, List<String>> {

        // validate weights
        require(weights.size == criteria.size) {
            "Weights length must match criteria length"
        }

        // validate prefs
        require(prefs.all { it in setOf("max", "min") }) {
            "Prefs must be 'max' or 'min'"
        }
        require(prefs.size == criteria.size) {
            "Prefs length must match criteria length"
        }

        // validate vetoes
        require(vetoes.size == criteria.size) {
            "Vetoes length must match criteria length"
        }

        // validate preference thresholds
        preferenceThresholds?.let {
            require(it.size == criteria.size) {
                "Preference thresholds length must match criteria length"
            }
        }

        // calculate matrices
        val (concordanceMatrix, nonDiscordanceMatrix) =
            getElectre1Matrices(weights, prefs, vetoes, preferenceThresholds)

        // calculate kernels
        val outrankingMatrix = getOutrankingMatrix(
            concordanceMatrix, nonDiscordanceMatrix,
            concordanceThreshold
        )
        val kernels = getKernels(outrankingMatrix)

        // robustness analysis
        val (robustnessAnalysisResults, frequentKernels) =
            robustnessAnalysis(concordanceMatrix, nonDiscordanceMatrix)

        // print output
        if (verbosePrintOutput) {
            printElectre(
                concordanceMatrix,
                nonDiscordanceMatrix,
                outrankingMatrix,
                kernels,
                robustnessAnalysisResults,
                frequentKernels
            )
        }

        // return results
        return mapOf(
            "kernels" to kernels,
            "frequentKernels" to frequentKernels
        )
    }
}

In [99]:
val data = listOf(
    listOf(80.0, 90.0, 600.0, 5.4, 8.0, 5.0),
    listOf(65.0, 58.0, 200.0, 9.7, 1.0, 1.0),
    listOf(83.0, 60.0, 400.0, 7.2, 4.0, 7.0),
    listOf(40.0, 80.0, 1000.0, 7.5, 7.0, 10.0),
    listOf(52.0, 72.0, 600.0, 2.0, 3.0, 8.0),
    listOf(94.0, 96.0, 700.0, 3.6, 5.0, 6.0),
)

val weights = listOf(0.1, 0.2, 0.2, 0.1, 0.2, 0.2)
val prefs = listOf("min", "max", "min", "min", "min", "max")
val vetoes = listOf(45.0, 29.0, 550.0, 6.0, 4.5, 4.5)
val indifference_threshold = 0.6
val preference_thresholds = listOf(20.0, 10.0, 200.0, 4.0, 2.0, 2.0) // or null for Electre Iv

In [100]:
val e = Electre(data, true)
e

Line_101_jupyter$Electre@45f7ceca

In [101]:
val results = e.solve(weights, prefs, vetoes, indifference_threshold, preference_thresholds)

concordanceMatrix: 
NaN		0.525		0.400		0.600		0.415		0.535
0.500		NaN		0.698		0.445		0.435		0.500
0.740		0.510		NaN		0.500		0.400		0.710
0.548		0.600		0.593		NaN		0.500		0.303
0.800		0.600		0.800		0.580		NaN		0.800
0.830		0.500		0.545		0.700		0.360		NaN

nonDiscordanceMatrix: 
NaN		0.000		1.000		0.000		0.000		1.000
0.000		NaN		0.000		0.000		0.000		0.000
0.000		1.000		NaN		1.000		1.000		0.000
1.000		0.000		0.000		NaN		1.000		1.000
1.000		1.000		1.000		1.000		NaN		1.000
1.000		1.000		1.000		0.000		1.000		NaN

outrankingMatrix: 
null		false		false		false		false		false
false		null		false		false		false		false
false		false		null		false		false		false
false		false		false		null		false		false
true		true		true		false		null		true
true		false		false		false		false		null

[4, 5]
Robustness Analysis:
[0.500] = 5
[0.525] = 5
[0.550] = 5
[0.575] = 5
[0.600] = 2, 4, 5
[0.625] = 2, 4, 5
[0.650] = 2, 4, 5
[0.675] = 2, 4, 5
[0.700] = 2, 4, 5
[0.725] = 2, 4, 5
[0.750] = 2, 4, 5
[0.775] = 2, 4, 5
[0.800] = 2,

In [102]:
// %use dataframe

In [103]:
val formattedResults = results.map { (key, value) ->
    "$key:\n${value.joinToString("\n")}"
}.joinToString("\n\n")

println("$formattedResults")

kernels:
4
5

frequentKernels:
5 (21)
2 (17)
4 (17)
3 (9)
6 (9)
1 (7)
